In [1]:
import sys

sys.path.insert(0, "../..")
sys.path.insert(0, "../../../")

# Load the Pretrained Model and the dataset

In [2]:
import numpy as np
import paddle
import paddlenlp
from paddlenlp.transformers import ErnieForSequenceClassification, ErnieTokenizer

MODEL_NAME = "ernie-1.0"

model = ErnieForSequenceClassification.from_pretrained(MODEL_NAME, num_classes=2)
tokenizer = ErnieTokenizer.from_pretrained(MODEL_NAME)

[2022-04-25 16:26:19,747] [    INFO] - Already cached /home/zhangshuai/.paddlenlp/models/ernie-1.0/ernie_v1_chn_base.pdparams
W0425 16:26:19.749972 14043 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.4, Runtime API Version: 10.2
W0425 16:26:19.755374 14043 device_context.cc:465] device: 0, cuDNN Version: 8.2.
[2022-04-25 16:26:24,488] [    INFO] - Already cached /home/zhangshuai/.paddlenlp/models/ernie-1.0/vocab.txt


In [3]:
from paddlenlp.datasets import load_dataset

DATASET_NAME = 'chnsenticorp'
train_ds, dev_ds, test_ds = load_dataset(DATASET_NAME, splits=["train", "dev", "test"])

# Prepare the Model
## Train the model

In [4]:
# training the model and save to save_dir
# only needs to run once.

from assets.utils import training_model

training_model(model, tokenizer, train_ds, dev_ds, save_dir=f'../../assets/{DATASET_NAME}-{MODEL_NAME}')

dataset labels: ['0', '1']
dataset examples:
{'text': '选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般', 'label': 1, 'qid': ''}
{'text': '15.4寸笔记本的键盘确实爽，基本跟台式机差不多了，蛮喜欢数字小键盘，输数字特方便，样子也很美观，做工也相当不错', 'label': 1, 'qid': ''}
{'text': '房间太小。其他的都一般。。。。。。。。。', 'label': 0, 'qid': ''}
{'text': '1.接电源没有几分钟,电源适配器热的不行. 2.摄像头用不起来. 3.机盖的钢琴漆，手不能摸，一摸一个印. 4.硬盘分区不好办.', 'label': 0, 'qid': ''}
{'text': '今天才知道这书还有第6卷,真有点郁闷:为什么同一套书有两种版本呢?当当网是不是该跟出版社商量商量,单独出个第6卷,让我们的孩子不会有所遗憾。', 'label': 1, 'qid': ''}
Training Starts:


In [4]:
# Load the trained model.
!wget --no-check-certificate -c https://trustai.bj.bcebos.com/chnsenticorp-ernie-1.0.tar
!tar -xvf ./chnsenticorp-ernie-1.0.tar -C ../../assets/
!rm ./chnsenticorp-ernie-1.0.tar

state_dict = paddle.load(f'../../assets/{DATASET_NAME}-{MODEL_NAME}/model_state.pdparams')
model.set_dict(state_dict)

--2022-04-25 16:26:24--  https://trustai.bj.bcebos.com/chnsenticorp-ernie-1.0.tar
Resolving trustai.bj.bcebos.com (trustai.bj.bcebos.com)... 10.70.0.165
Connecting to trustai.bj.bcebos.com (trustai.bj.bcebos.com)|10.70.0.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 399595520 (381M) [application/x-tar]
Saving to: ‘chnsenticorp-ernie-1.0.tar’

100%[======================================>] 399,595,520  120MB/s   in 3.2s   

2022-04-25 16:26:27 (120 MB/s) - ‘chnsenticorp-ernie-1.0.tar’ saved [399595520/399595520]

chnsenticorp-ernie-1.0/
chnsenticorp-ernie-1.0/tokenizer_config.json
chnsenticorp-ernie-1.0/vocab.txt
chnsenticorp-ernie-1.0/model_state.pdparams
chnsenticorp-ernie-1.0/model_config.json


# See the prediction results

In [5]:
from assets.utils import predict

data = [
    {
        "text": '这个宾馆比较陈旧了，特价的房间也很一般。总体来说一般'
    },
    {
        "text": '怀着十分激动的心情放映，可是看着看着发现，在放映完毕后，出现一集米老鼠的动画片'
    },
    {
        "text": '作为老的四星酒店，房间依然很整洁，相当不错。机场接机服务很好，可以在车上办理入住手续，节省时间。'
    },
]

label_map = {0: 'negative', 1: 'positive'}
true_labels = [0, 0, 1]
batch_size = 32
results = predict(model, data, tokenizer, label_map, batch_size=batch_size)

for idx, text in enumerate(data):
    print('Data: {} \t Label: {}'.format(text, results[idx]))

Data: {'text': '这个宾馆比较陈旧了，特价的房间也很一般。总体来说一般'} 	 Label: negative
Data: {'text': '怀着十分激动的心情放映，可是看着看着发现，在放映完毕后，出现一集米老鼠的动画片'} 	 Label: negative
Data: {'text': '作为老的四星酒店，房间依然很整洁，相当不错。机场接机服务很好，可以在车上办理入住手续，节省时间。'} 	 Label: positive


Prepare for Interpretations

In [6]:
from paddlenlp.data import Stack, Tuple, Pad

from assets.utils import convert_example
from trustai.interpretation.token_level.data_processor import VisualizationTextRecord, visualize_text

def preprocess_fn(data):
    examples = []

    if not isinstance(data, list):
        data = [data]

    for text in data:
        input_ids, segment_ids = convert_example(text, tokenizer, max_seq_length=128, is_test=True)
        examples.append((input_ids, segment_ids))

    batchify_fn = lambda samples, fn=Tuple(
        Pad(axis=0, pad_val=tokenizer.pad_token_id),  # input id
        Pad(axis=0, pad_val=tokenizer.pad_token_id),  # segment id
    ): fn(samples)

    input_ids, segment_ids = batchify_fn(examples)
    return paddle.to_tensor(input_ids, stop_gradient=False), paddle.to_tensor(segment_ids, stop_gradient=False)

In [7]:
import jieba

from trustai.interpretation import get_word_offset

contexts = []
batch_words = []
for example in data:
    contexts.append("[CLS]" + " " + example['text'] + " " + "[SEP]")
    batch_words.append(["[CLS]"] + list(jieba.cut(example['text'])) + ["[SEP]"])
word_offset_maps = []
subword_offset_maps = []
for i in range(len(contexts)):
    word_offset_maps.append(get_word_offset(contexts[i], batch_words[i]))
    subword_offset_maps.append(tokenizer.get_offset_mapping(contexts[i]))

print("word_offset_map:")
for word, (offset_start, offset_end) in zip(batch_words[0], word_offset_maps[0]):
    print(word, offset_start, offset_end)
print("subword_offset_map:")
for subword, (offset_start, offset_end) in zip(tokenizer._tokenize(contexts[0]), subword_offset_maps[0]):
    print(subword, offset_start, offset_end)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Dumping model to file cache /tmp/jieba.cache
Dump cache file failed.
Traceback (most recent call last):
  File "/home/zhangshuai/miniconda3/envs/zs_py39/lib/python3.9/site-packages/jieba/__init__.py", line 154, in initialize
    _replace_file(fpath, cache_file)
PermissionError: [Errno 1] Operation not permitted: '/tmp/tmpqo33i7a4' -> '/tmp/jieba.cache'
Loading model cost 0.879 seconds.
Prefix dict has been built successfully.
word_offset_map:
[CLS] 0 5
这个 6 8
宾馆 8 10
比较 10 12
陈旧 12 14
了 14 15
， 15 16
特价 16 18
的 18 19
房间 19 21
也 21 22
很 22 23
一般 23 25
。 25 26
总体 26 28
来说 28 30
一般 30 32
[SEP] 33 38
subword_offset_map:
[ 0 1
cl 1 3
##s 3 4
[UNK] 4 5
这 6 7
个 7 8
宾 8 9
馆 9 10
比 10 11
较 11 12
陈 12 13
旧 13 14
了 14 15
， 15 16
特 16 17
价 17 18
的 18 19
房 19 20
间 20 21
也 21 22
很 22 23
一 23 24
般 24 25
。 25 26
总 26 27
体 27 28
来 28 29
说 29 30
一 30 31
般 31 32
[ 33 34
sep 34 37
[UNK] 37 38


# IG Interpreter

In [8]:
from trustai.interpretation.token_level import IntGradInterpreter
ig = IntGradInterpreter(model)
result = ig(preprocess_fn(data), steps=100)

align_res = ig.alignment(result, contexts, batch_words, word_offset_maps, subword_offset_maps, special_tokens=["[CLS]", '[SEP]'])

# process for vbisualize
recs = []
for i in range(len(align_res)):
    recs.append(VisualizationTextRecord(align_res[i], true_label=true_labels[i]))
html = visualize_text(recs)


# print interpret result

In [9]:
print("interpret result of first example:")
for field in result[0].__dataclass_fields__:
    print(field, ":", getattr(result[0], field))
print("alignment result of first example:")
for field in align_res[0].__dataclass_fields__:
    print(field, ":", getattr(align_res[0], field))


interpret result of first example:
attributions : [ 0.02149865  0.13750568  0.03729623  0.20981199  0.11474895  0.00191162
  0.01166647  0.01939347  0.00499799 -0.01771647  0.05467343 -0.05574901
  0.0797711   0.02094495 -0.02661019  0.01423277  0.03983632  0.05040766
  0.03474617  0.10548145 -0.02475511 -0.06759283 -0.07004125 -0.0207927
  0.03771218  0.01511401 -0.01349011  0.01542336  0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.        ]
pred_label : 0
pred_proba : [0.86797816 0.1320218 ]
error_percent : -2.2929103
alignment result of first example:
words : ['[CLS]', '这个', '宾馆', '比较', '陈旧', '了', '，', '特价', '的', '房间', '也', '很', '一般', '。', '总体', '来说', '一般', '[SEP]']
word_attributions : [0.021498650312423706, 0.17480190843343735, 0.3245609328150749, 0.013578088022768497, 0.02439146302640438, -0.017716471105

# LIME Interpreter

In [10]:
from trustai.interpretation.token_level import LIMEInterpreter
lime = LIMEInterpreter(model,
    unk_id=tokenizer.convert_tokens_to_ids('[UNK]'),
    pad_id=tokenizer.convert_tokens_to_ids('[PAD]'))

result = lime(preprocess_fn(data), num_samples=1000)
align_res = lime.alignment(result, contexts, batch_words, word_offset_maps, subword_offset_maps, special_tokens=["[CLS]", '[SEP]'])

# process for vbisualize
recs = []
for i in range(len(align_res)):
    recs.append(VisualizationTextRecord(align_res[i], true_label=true_labels[i]))
html = visualize_text(recs)

# Attention Interpreter

In [11]:
from trustai.interpretation.token_level.common import attention_predict_fn_on_paddlenlp
from trustai.interpretation.token_level import AttentionInterpreter

att = AttentionInterpreter(model, predict_fn=attention_predict_fn_on_paddlenlp)

result = att(preprocess_fn(data))
align_res = att.alignment(result, contexts, batch_words, word_offset_maps, subword_offset_maps, special_tokens=["[CLS]", '[SEP]'])

# process for vbisualize
recs = []
for i in range(len(align_res)):
    recs.append(VisualizationTextRecord(align_res[i], true_label=true_labels[i]))
html = visualize_text(recs)

# GradShap Interpreter

In [12]:
from trustai.interpretation.token_level import GradShapInterpreter
gradshap = GradShapInterpreter(model, n_samples=50, noise_amount=0.1)
result = gradshap(preprocess_fn(data))
align_res = gradshap.alignment(result,
                               contexts,
                               batch_words,
                               word_offset_maps,
                               subword_offset_maps,
                               special_tokens=["[CLS]", '[SEP]'])

# process for vbisualize
recs = []
for i in range(len(align_res)):
    recs.append(VisualizationTextRecord(align_res[i], true_label=true_labels[i]))
html = visualize_text(recs)